# IMPORT

In [1]:
!pip install seaborn==0.10.1
!pip install -U scikit-learn

    100% |████████████████████████████████| 225kB 13.7MB/s ta 0:00:01
  Found existing installation: seaborn 0.8.1
    Uninstalling seaborn-0.8.1:
      Successfully uninstalled seaborn-0.8.1
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 6.9MB 7.1MB/s eta 0:00:01
    100% |████████████████████████████████| 307kB 39.7MB/s ta 0:00:01
Requirement not upgraded as not directly required: scipy>=0.19.1 in /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages (from scikit-learn) (1.1.0)
Requirement not upgraded as not directly required: numpy>=1.13.3 in /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages (from scikit-learn) (1.14.6)
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3
You are using pip version 10.0.1, h

In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import psutil
import re

# magic word for producing visualizations in notebook
%matplotlib inline

# CUSTOM FUNCTIONS

In [3]:
def remove_high_corr(corr, threshold, output_high_corr=False):
    """
    Only  keep one element from a chain of highly (> threshold) correlated elements
    
    Parameters:
    -----------
    corr (pandas.DataFrame) : the correlation matrix
    threshold (float) : the threshold above which a high correlation is considered between elements
    
    Returns:
    --------
    if not output_high_corr:
    (main_elements, correlated_elements) (tuple) : a tuple containing two lists featuring respectively the main elements (to be kept)
                                                   and the correlated elements with the main elements (to be discarded)
    if output_high_corr:
    (-, -, high_corr) : same as above + high_corr which provides a dictionnary providing pairs of highly correlated variables 
    
    """
    
    indices = np.where(corr > threshold)
    high_corr = {corr.index[x] : corr.columns[y] for x, y in zip(*indices)
                                        if x != y and x < y}
    main_elements = []
    correlated_elements = []
    
    for key in high_corr.keys():
        if (key not in main_elements) and (key not in correlated_elements):
            main_elements.append(key)
            correlated_elements.append(high_corr[key])
        elif key in correlated_elements:
            correlated_elements.append(high_corr[key])
        else:
            continue
    
    if not output_high_corr:
        return (main_elements, correlated_elements)
    else:
        return (main_elements, correlated_elements, high_corr)
        

In [4]:
def drop_full_empty_row(df, df_name=None):
    """
    Drop completely empty rows from  a dataframe
    
    Parameters:
    -----------
    df (pandas.DataFrame) : DataFrame to be processed
    df_name (str) : a human-understandble name of the dataframe for messaging about operations
    
    Returns:
    --------
    None
    """
    if not df_name:
        df_name = "DataFrame"
        
    print(f'Before deletion of full empty rows, we have {len(df)} samples in {df_name} data')
    df.dropna(axis=0, how='all')
    print(f'After deletion of full empty rows, we have {len(df)} samples in {df_name} data')   


In [5]:
def remove_insignificant_columns(df, thresh = 0.8):
    """
    Removes columns where NaN counts / total values exceed thresh
    
    Parameters:
    -----------
    df (pandas.DataFrame) : DataFrame to be processed
    thresh (float) : threshold for deleting columns
    
    Returns:
    --------
    list of removed columns
    
    """
    removed_col = []
    
    total_count  = df.shape[0]
    
    for col in df.columns:
        nan_count = df[col].isna().sum()
        if nan_count/total_count > thresh:
            df.drop(col, axis=1, inplace=True)
            print(f'column {col} has been dropped')
            removed_col.append(col)
    
    return removed_col

In [6]:
def construct_fill_na(filename, df):
    """
    Construct a dataframe identifying nan values for a dataframe that differ from np.nan
    Construct a dictionnary providing replacements proposals in case multiple equivalent nan.values are identified for same feature
    
    Parameters:
    -----------
    filename (pathlib.Path or str) : path to the filename containing the information about NaN values
                                     ! current function only works on a very specific template for this file
    
    df (pandas.DataFrame) : dataframe for which the nan values must be identified
    
    Returns:
    --------
    nan_info (pd.DataFrame) : dataframe identifying which values in the dataframe df are equivalent to np.nan
    replacements (dict) : a dictionnary whose keys are (some) of the columns of df and whose values are dictionnaries 
                         providing {value_to_be_replaced : value_to_replace}

    
    """    
    
    
    nan_info = pd.read_excel(filename,
                        usecols=[1,3,4])
    nan_info.columns = ["Attribute", "Value", "Meaning"]
    nan_info.fillna(method='ffill', axis=0, inplace=True)

    # store index of lines containing "unknown" levels
    target_index = []
    for i, row in nan_info.iterrows():
        try:
            if "unknown" in row.iloc[-1]:
                target_index.append(i)
        except:
            continue
    
    nan_info = nan_info.iloc[target_index, [0,1]]
    nan_info.set_index("Attribute", inplace=True) # index provide attribute, corresponding value is the NaN value for that attribute
    
    # some attributes have two possible values
    # identify which ones, consider only one value and make
    # the dataframe pop_df & customer_df consistent with the one considered

    replacements = {}
    for i, row in nan_info.iterrows():
        if type(row.values[0]) == str:
            if len(row.values[0].split()) > 1:
                kept, dropped = row.values[0].split()
                nan_info.loc[i] = kept
                replacements[i] = {dropped:kept} # this will be used to replace in original dataframe
    
    # for some replacements, it may happen that a str is read
    # altough the corresponding column in dataframe contains float 
    # so for each column that is dtype numeric, remove anything that could hinder a 
    # conversion from string to float e.g. commas
    for col in df.select_dtypes(include=[np.number]):
        if col in nan_info.index.values:
            if isinstance(nan_info.loc[col,:].values[0], str):
                nan_info.loc[col,:] = float(nan_info.loc[col,:].values[0].replace(',',''))
                if col in replacements.keys():
                    for key, value in replacements[col].items():
                        replacements[col] = {float(key.replace(',','')) : float(value.replace(',',''))}

    for row in nan_info.index:
        if row in df.columns:
            if df[row].dtype != 'object':
                nan_info.loc[row,:] = float(nan_info.loc[row,:].values) # make sure the fill_na value type matches corresponding column dtype in original dataframe
                                                     # everything can be set to float except 'object' dtype for which fill_na value which must remain string, already the case
                          
                    
                      
    return nan_info, replacements

In [7]:
def make_replacement(df, replacement):
    """
    replace values in a dataframe according to a dictionnary

    Parameters:
    -----------
    df (pandas.DataFrame) : the dataframe on which replacements must be made
    replace (dict) : a dictionnary whose keys are (some) of the columns of df and whose values are dictionnaries 
                     providing {value_to_be_replaced : value_to_replace}

    Returns:
    --------
    df (pandas.DataFrame) : modified dataframe with replacements performed

    """
    
    count = 0
    
    for col in df.columns:
        if col in replacements.keys():
            try:
                df.loc[:,col] = df.loc[:,col].replace(replacements[col])
                count+=1
            except Exception as e:
                print(e)
    
    print(f'{count} replacements made')
    
    return df

In [8]:
def fill_na_presc(df, nan_fill):
    """
    replace equivalent NaN identified in nan_fill by np.nan
    
    Parameters:
    -----------
    df (pandas.DataFrame) : Dataframe on which NaN will be replaced
    nan_fill (pandas.DaTaframe) : DataFrame providing the rules for NaN replacement
    
    Returns:
    --------
    None
    
    Remarks:
    --------
    nan_fill shall have its index corresponding to columns in df in which one desires to replace equivalent NaN values
    nan_fill.loc[col, "Value"] shall provide the equivalent NaN value in column col of df to be replaced by np.nan
    ! TODO : make it generic and parameterized

    """
    for col in df.columns :
        if col in nan_fill.index:
            try:
                #df.loc[df[col].isna()==True, col] = nan_fill.loc[col, "Value"]
                df.loc[:,col] = df.loc[:,col].replace(nan_fill.loc[col, "Value"], np.nan) # inplace replace is buggy, don't use
            except Exception as e:
                if "Cannot setitem" in str(e):
                    # if no unknown category yet in that column, add the value to the categories
                    df[col].cat.set_categories(np.hstack((df[col].cat.categories.values,
                                     np.nan)), inplace=True)
                    df.loc[:,col] = df.loc[:,col].replace(nan_fill.loc[col, "Value"], np.nan) # inplace replace is buggy, don't use
                else:
                    print(e)

In [9]:
def identify_numeric(filename, df):
    """
    Identify numeric columns based on information contained in filename
    
    Parameters:
    -----------
    filename (pathlib.Path or str) : path to the filename containing the information about numeric values
                                     ! current function only works on a very specific template for this file
        
    Returns:
    --------
    a list providing the name of numeric columns

    
    """    
    
    
    num_info = pd.read_excel(filename,
                        usecols=[1,3,4])
    num_info.columns = ["Attribute", "Value", "Meaning"]
    num_info.fillna(method='ffill', axis=0, inplace=True)

    # store index of lines containing "numeric" levels
    target_index = []
    for i, row in num_info.iterrows():
        try:
            if "numeric" in row.iloc[-1]:
                target_index.append(i)
        except:
            continue
    
    num_info = num_info.iloc[target_index, [0,1]]
    num_info.set_index("Attribute", inplace=True) # index provide attribute, corresponding value is the NaN value for that attribute
    

                      
    return list(num_info.index)

In [10]:
def df_to_numeric(df):
    column_not_converted = []
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='raise')
        except Exception as e:
            print(f'column {col} could not be converted to numeric')
            column_not_converted.append(col)
    return column_not_converted

In [11]:
def get_non_numeric(df):
    non_numeric = []
    for col in df.columns:
        if df[col].dtype not in (np.float64, np.int64):
            non_numeric.append(col)
    return non_numeric

In [12]:
def non_frequent_to_nan(df, threshold = 0.02):
    """
    replace non frequent (lower than threshold) occurences inside df with np.nan
    
    Parameters:
    -----------
    df (pandas.DataFrame) : Dataframe in which non-frequent occurences will be replaced
    thresh (float) : threshold for non-frequent categorization

        
    Returns:
    --------
    df with non-frequent occurences replaced with np.nan

    
    """    
    for col in df.columns:
        tot_values = df[col].shape[0]
        value_counts = df[col].value_counts()
        to_remove = value_counts[value_counts/tot_values <= threshold].index
        df[col] = df[col].replace(to_remove, np.nan) # replace inplace buggy

In [13]:
def split_cameo(df, column):
    """
    split column into two columns with separate information
    
    Parameters:
    -----------
    df (pandas.DataFrame) : Dataframe in which non-frequent occurences will be replaced
    column (str) : name of column to be split

        
    Returns:
    --------
    df with column split

    
    """    
    
    def spit_content(row):
        feat1, feat2 = list(str(int(pop_df_selected['CAMEO_INTL_2015'].iloc[0])))
        return [int(feat1), int(feat2)]
        
        
    columns = df[column].apply(spit_content)
    columns = pd.DataFrame(columns.tolist(), index= df.index, columns=["CAMEO1", "CAMEO2"])
    df = df.join(columns)
    df.drop(column, axis=1, inplace=True)
    
    return df

# DATA STORAGE TO S3

In [14]:
import sagemaker
session = sagemaker.session.Session
bucket = 'auto-ml-exploration'

In [13]:
s3_dataset_path = f's3://{bucket}/dataset'

sagemaker.s3.S3Uploader.upload('Udacity_AZDIAS_052018.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_CUSTOMERS_052018.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_MAILOUT_052018_TEST.csv', s3_dataset_path)
sagemaker.s3.S3Uploader.upload('Udacity_MAILOUT_052018_TRAIN.csv', s3_dataset_path)

# LOAD DATA

## Data location

In [15]:
# data location on S3
pop_dataset_loc = f's3://{bucket}/dataset/Udacity_AZDIAS_052018.csv'
customers_dataset_loc = f's3://{bucket}/dataset/Udacity_CUSTOMERS_052018.csv'

## Partial loading

In [16]:
# partial loading
# load in the data, first only a few rows to get an idea
# I had a memory issue when loading the complete file
# 1 every 2 rows
pop_df_partial = pd.read_csv(pop_dataset_loc, sep=';', skiprows = lambda x: x%2) # skiprows because much data... Let's see if one can convert some to categorical to reduce memory usage

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
customers_df_partial = pd.read_csv(customers_dataset_loc, sep=';', skiprows = lambda x: x%2)

## Mini loading

In [ ]:
pop_df_mini = pop_df_partial.iloc[0:100,:]
customers_df_partial = customers_df_partial.iloc[0:100,:]

## Full loading

In [ ]:
pop_df = pd.read_csv(pop_dataset_loc, sep=';', dtype = cat_col)
cat_col_customers = cat_col.copy()
cat_col_customers['CUSTOMER_GROUP'] = 'category'
cat_col_customers['ONLINE_PURCHASE'] = 'category'
cat_col_customers['PRODUCT_GROUP'] = 'category'

customers_df = pd.read_csv(customers_dataset_loc, sep=';', dtype = cat_col_customers)

## Data selection

In [17]:
pop_df_selected = pop_df_partial

In [ ]:
customers_df_selected = customers_df_partial

## Get features names

In [18]:
customers_categories = customers_df_selected.columns
pop_categories = pop_df_selected.columns

NameError: name 'customers_df_selected' is not defined

# Clean Data

Following data cleaning is performed based on the insights gained from data exploration notebook

## Setting a better index

In [20]:
pop_df_selected.set_index('LNR', inplace=True)

In [21]:
pop_df_selected.head()

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910220,NaN,9.0,NaN,NaN,NaN,NaN,NaN,21.0,11.0,0.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5.0,2.0,1.0
910226,2.0,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,0.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3.0,2.0,4.0
910244,3.0,1.0,10.0,NaN,NaN,NaN,NaN,10.0,5.0,0.0,...,2.0,10.0,7.0,4.0,6.0,9.0,7.0,4.0,2.0,1.0
910261,NaN,1.0,14.0,NaN,NaN,NaN,NaN,14.0,6.0,0.0,...,2.0,10.0,12.0,9.0,5.0,9.0,1.0,1.0,1.0,1.0
645153,NaN,5.0,17.0,NaN,NaN,NaN,NaN,17.0,9.0,0.0,...,4.0,1.0,1.0,1.0,4.0,3.0,7.0,4.0,2.0,3.0


## Droping columns based on user choice

In [22]:
# specify columns to be dropped
pop_columns_to_drop = ['CAMEO_DEUG_2015', 'CAMEO_DEU_2015', 'VERDICHTUNGSRAUM', 
                       'ALTERSKATEGORIE_FEIN','EINGEFUEGT_AM', 'LP_LEBENSPHASE_FEIN']
customers_columns_to_drop = pop_columns_to_drop + ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']

# explanations
# 'CAMEO_DEUG_2015', 'CAMEO_DEU_2015' are dropped because they are redundant regarding given information with 'CAMEO_DEU_INTL'
# 'VERDICHTUNGSRAUM' : many 0s (supposedly = unkown), many categories (impairing one hot encoding)
# 'ALTERSKATEGORIE_FEIN' : Age category Fine ... Similar to ALTER_HH from my understanding
# 'EINGEFUEGT_AM' : seems to be the time of input of the data, many categories
# 'LP_LEBENSPHASE_FEIN' : provide similar info as CAMEO & many categories

In [23]:
pop_df_selected.drop(pop_columns_to_drop, axis=1, inplace=True, errors='ignore')
#customers_df_selected.drop(customers_columns_to_drop, axis=1, inplace=True, errors='ignore')

## Dropping full empty rows

In [24]:
drop_full_empty_row(pop_df_selected, 'population dataframe')
#drop_full_empty_row(customers_df_selected, 'customers dataframe')

Before deletion of full empty rows, we have 363212 samples in population dataframe data
After deletion of full empty rows, we have 363212 samples in population dataframe data


## Remove insignificant columns (i.e. many NaN)

In [25]:
remove_insignificant_columns(pop_df_selected, thresh = 0.60) # threshold based on exploratory analysis
#remove_insignificant_columns(customers_df_selected)

column AGER_TYP has been dropped
column ALTER_KIND1 has been dropped
column ALTER_KIND2 has been dropped
column ALTER_KIND3 has been dropped
column ALTER_KIND4 has been dropped
column EXTSEL992 has been dropped
column KK_KUNDENTYP has been dropped
column TITEL_KZ has been dropped


['AGER_TYP',
 'ALTER_KIND1',
 'ALTER_KIND2',
 'ALTER_KIND3',
 'ALTER_KIND4',
 'EXTSEL992',
 'KK_KUNDENTYP',
 'TITEL_KZ']

## Droping highly correlated columns

In [26]:
corr = pop_df_selected.corr()
(main_elements, correlated_elements) = remove_high_corr(corr, 0.7)
pop_df_selected.drop(correlated_elements, axis=1, inplace=True)

# corr = customers_df_selected.corr()
# (main_elements, correlated_elements) = remove_high_corr(corr, 0.7)
# customers_df_selected.drop(correlated_elements, axis=1,inplace=True)

## Handling objects columns

In [27]:
object_columns = pop_df_selected.select_dtypes('object').columns
pop_df_selected.loc[:,object_columns] = pop_df_selected.loc[:,object_columns].replace('[X]+', '99942', regex=True) #99942 will be used as a flag for NaN
                                                                                         # mandatory as regex cannot replace by non-string
                                                                                         # inplace not used because not working (apparently, bug according to SO with mixed data)
not_converted = df_to_numeric(pop_df_selected)
pop_df_selected.loc[:,not_converted] = pop_df_selected.loc[:,not_converted].astype('category')

column D19_LETZTER_KAUF_BRANCHE could not be converted to numeric
column OST_WEST_KZ could not be converted to numeric


In [28]:
# replacing nan placeholder immediately
pop_df_selected = pop_df_selected.replace(99942, np.nan)

## Finding NaN equivalent and converting to np.nan

In [29]:
filename= 'DIAS Attributes - Values 2017.xlsx'
nan_info, replacements = construct_fill_na(filename, pop_df_selected) # find nan equivalent
make_replacement(pop_df_selected, replacements) # make dataframe consistent if multiple nan equivalent for same feature
fill_na_presc(pop_df_selected, nan_info) # replace nan equivalent by np.nan


75 replacements made


## Removing insignificant columns - again, after cleaning

In [30]:
remove_insignificant_columns(pop_df_selected, thresh = 0.60)

[]

## Splitting CAMEO

In [31]:
copy = pop_df_selected.copy()

In [33]:
pop_df_selected = split_cameo(pop_df_selected, 'CAMEO_INTL_2015') # more efficient regarding one hot encoding - less columns added

## Now Moving to scikit -> column & index will be lost since scikit works with arrays

In [46]:
# remember columns & index
pop_columns = pop_df_selected.columns
# customers_columns = customers_df_selected.columns

pop_index = pop_df_selected.index
# customers_index = customers_df_selected.index


## Replacing NaN with most frequent value

In [43]:
from sklearn.impute import SimpleImputer

imp_frequent = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
pop_df_selected = imp_frequent.fit_transform(pop_df_selected)

## Identifying categorical vs numeric for further post-treatment

In [52]:
pop_df_numeric = identify_numeric(filename, pop_df_selected) # based on Excel file
pop_df_cat = [col for col in pop_df_selected.columns if col not in pop_df_numeric]

In [58]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder

numeric_pipeline = make_pipeline(Normalizer())
categorical_pipeline = make_pipeline(OneHotEncoder())

In [59]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('categorical_transformer', categorical_pipeline, pop_df_cat),
                                ('numeric_transformer', numeric_pipeline, pop_df_numeric)
                                    ]
                      )

In [60]:
pop_df_selected = ct.fit_transform(pop_df_selected)

In [61]:
pop_df_selected.shape

(363212, 1755)

In [62]:
copy

,AKT_DAT_KL,ALTER_HH,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_INTL_2015,...,UMFELD_JUNG,UNGLEICHENN_FLAG,VERS_TYP,VHA,VHN,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910220,9.0,NaN,11.0,0.0,0.0,2.0,0.0,3.0,6.0,51.0,...,3.0,1.0,2.0,0.0,4.0,3.0,9.0,4.0,5.0,1.0
910226,1.0,13.0,1.0,0.0,0.0,0.0,0.0,2.0,4.0,12.0,...,5.0,0.0,1.0,1.0,0.0,NaN,9.0,7.0,3.0,4.0
910244,1.0,10.0,5.0,0.0,0.0,1.0,0.0,2.0,6.0,54.0,...,3.0,0.0,2.0,0.0,2.0,6.0,9.0,7.0,4.0,1.0
910261,1.0,14.0,6.0,0.0,0.0,1.0,0.0,2.0,2.0,14.0,...,4.0,0.0,1.0,0.0,2.0,5.0,9.0,1.0,1.0,1.0
645153,5.0,17.0,9.0,0.0,0.0,1.0,0.0,2.0,6.0,15.0,...,5.0,0.0,2.0,0.0,4.0,4.0,3.0,7.0,4.0,3.0
645169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
612561,8.0,20.0,2.0,0.0,0.0,1.0,0.0,3.0,7.0,33.0,...,3.0,0.0,2.0,0.0,NaN,6.0,3.0,5.0,5.0,1.0
612569,9.0,11.0,1.0,0.0,0.0,1.0,0.0,4.0,1.0,41.0,...,2.0,0.0,2.0,0.0,4.0,6.0,4.0,3.0,3.0,4.0
612577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0


In [54]:
pop_df_selected.drop(['ALTERSKATEGORIE_FEIN', 'EINGEFUEGT_AM', 'EINGEZOGENAM_HH_JAHR', 'VERDICHTUNGSRAUM'], axis=1, inplace=True)

In [51]:
[col for col in pop_df_selected.columns if 'LEBEN' in col]

['D19_LEBENSMITTEL', 'LP_LEBENSPHASE_FEIN']

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

# EXPERIMENTATION

In [38]:
pop_df_selected.isna().sum().sum()

10115643

In [47]:
pop_df_selected = pd.DataFrame(result, index=pop_index, columns=pop_columns)

In [48]:
pop_df_selected

,AKT_DAT_KL,ALTER_HH,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,CJT_GESAMTTYP,...,VERS_TYP,VHA,VHN,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ALTERSKATEGORIE_GROB,CAMEO1,CAMEO2
LNR,,,,,,,,,,,,,,,,,,,,,
910220,9,18,11,0,0,2,0,3,6,5,...,2,0,4,3,9,4,5,1,5,1
910226,1,13,1,0,0,0,0,2,4,2,...,1,1,0,6,9,7,3,4,5,1
910244,1,10,5,0,0,1,0,2,6,2,...,2,0,2,6,9,7,4,1,5,1
910261,1,14,6,0,0,1,0,2,2,3,...,1,0,2,5,9,1,1,1,5,1
645153,5,17,9,0,0,1,0,2,6,4,...,2,0,4,4,3,7,4,3,5,1
645169,1,18,1,0,0,1,0,4,6,6,...,2,0,2,6,9,3,3,2,5,1
612561,8,20,2,0,0,1,0,3,7,5,...,2,0,2,6,3,5,5,1,5,1
612569,9,11,1,0,0,1,0,4,1,4,...,2,0,4,6,4,3,3,4,5,1
612577,1,18,1,0,0,1,0,4,6,6,...,2,0,2,6,9,3,3,2,5,1
